In [ ]:
import pandas as pd
import py2neo
from py2neo import Graph, Node
graph = Graph(password="neo4j1")

import pandas as pd
import sqlite3
import pdfkit as pdf
from PyPDF2 import PdfFileMerger

In [ ]:
df = pd.read_csv('C:\\Users\\MEI3KOR\\Desktop\\ccent.csv')
df = df.fillna(' ')
df = df.drop_duplicates()

In [ ]:
ErrorMessages = []
for row in df.iterrows():
    sourcename     = row[1][0]
    sourceclass    = row[1][1]
    preconditions  = row[1][2]
    postconditions = row[1][3]
    valuerange     = row[1][4]
    constraints    = row[1][5]
    states         = row[1][6]
    defaultstate   = row[1][7]
    attributes     = row[1][8]
    
    x = "MATCH (source:" + sourceclass + " {name:" +"'"+ sourcename+"'" + "}) RETURN source.name"
    xexist = graph.run(x)
    lenx = len(list(xexist))
    if lenx > 0:
        if lenx > 1:
            Message = "Entity:" +sourcename+' '+ "with Entity Class:"+sourceclass +' '+"has duplicate nodes in Knowledge Base"
        else:
            Message = "Entity:" +sourcename+' '+ "with Entity Class:"+sourceclass +' '+"exist in Knowledge Base"
    else:
#     if len(list(xexist)) == 0:
            x = "CREATE (n:"+ sourceclass +"{name:"+"'"+sourcename+"',"+"preconditions:"+"'"+preconditions+"',"+"postconditions:"+"'"+postconditions+"',"+"valuerange:"+"'"+valuerange+"',"+"constraints:"+"'"+constraints+"',"+"states:"+"'"+states+"',"+"defaultstate:"+"'"+defaultstate+"',"+"attributes:"+"'"+attributes+"'})"
            xcreate = graph.run(x)
            Message = "Entity:" +sourcename+' '+ "with Entity Class:"+sourceclass +' '+"created in Knowledge Base"
    ErrorMessages.append(Message)    

Messagedf = pd.DataFrame(ErrorMessages, columns=['Error Messages'])
if len(Messagedf.index) > 0:
    path_wkhtmltopdf = b'C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe'
    config = pdf.configuration(wkhtmltopdf=path_wkhtmltopdf)
    old_width = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', -1)
    Messagedf.to_html('C:\\Users\\MEI3KOR\\Desktop\\ht.html')
    pd.set_option('display.max_colwidth', old_width)
    pdf.from_file('C:\\Users\\MEI3KOR\\Desktop\\ht.html', 'C:\\Users\\MEI3KOR\\Desktop\\ErrorMessages.pdf', configuration=config)


    

In [ ]:
df = pd.read_csv('C:\\Users\\MEI3KOR\\Desktop\\one2.csv')

In [ ]:
for row in df.iterrows():
    sourcename    = row[1][0]
    targetname    = row[1][4]
    sourceclass   = row[1][1]
    targetclass   = row[1][5]
    relationname  = row[1][2]
    relationclass = row[1][3]
    propertyrc    = row[1][6]
    propertyr     = row[1][7]
    
    x1 = "MATCH (n)-[r:"+relationclass+"{relation:"+"'"+relationname+"'"+"}]->(m) return r" 
    x2 = "MATCH (n:"+sourceclass+")-[r:"+relationclass+"{relation:"+"'"+relationname+"'"+"}]->(m:"+targetclass+") return r"
    x3 = "MATCH (n:"+sourceclass+"{name:"+"'"+sourcename+"'"+"})-[r:"+relationclass+"{relation:"+"'"+relationname+"'"+"}]->(m:"+targetclass+"{name:"+"'"+targetname+"'"+"}) return r" 
    s3 = "MATCH (n:"+sourceclass+"{name:"+"'"+sourcename+"'"+"})-[r:"+relationclass+"{relation:"+"'"+relationname+"'"+"}]->(m:"+targetclass+"{name:"+"'"+targetname+"'"+"}) set r.propertyrc ="+"'"+propertyrc+"'"+",r.propertyr="+"'"+propertyr+"'" 
    s4 = "MATCH (n:"+sourceclass+")-[r:"+relationclass+"{relation:"+"'"+relationname+"'"+"}]->(m:"+targetclass+") set r.propertyrc ="+"'"+propertyrc+"'"
    
    resultx1 = graph.run(x1)
    if len(list(resultx1)) > 0:
        resultx2 = graph.run(x2)
        if len(list(resultx2)) > 0:
            resultx3 = graph.run(x3) 
            if len(list(resultx3)) > 0:
                 updater = graph.run(s3)
            else:
                 updaterc = graph.run(s4)
        